In [1]:
#Use this script to create a chat engine based on your personal documents with GPT-4!
#This code has been generated and tested in April 2024. It will not be modified after publication.
#If you use this for future work, please cite us. Thank you!
#(c) Hannah Muti, EKFZ for Digital Health, Dresden University of Technology

In [2]:
import os
import openai
openai.api_key = "YOUR-API-KEY" #Insert your OpenAI API key here. Alternatively, create an environmental variable using dotenv.

In [3]:
#This code uses the llama-index framework. If you run into errors, please check their documentation for updates and changes.
import llama_index
import chromadb

In [4]:
#Specify your embedding function to create vector embeddings of your documents.  
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

import chromadb.utils.embedding_functions as embedding_functions
embedding_function = embedding_functions.OpenAIEmbeddingFunction(
                api_key="YOUR-API-KEY",
                model_name="text-embedding-3-large"
            )

In [5]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

In [ ]:
#Read your documents. Ideally, store them within your project folder in a subfolder named "data" or edit the path accordingly.
documents = SimpleDirectoryReader("./data").load_data()
documents

In [7]:
# Initialize the chroma client to create your embeddings and set the path to store them
db = chromadb.PersistentClient(path="./SOP_vector_database")

In [8]:
#Get or create (the function can do both) your vector collection with the chroma client and the embedding function you initialized above. We use cosine similarity for similarity search.
chroma_collection = db.get_or_create_collection(name = "SOP_vector_database", embedding_function = embedding_function, metadata={"hnsw:space": "cosine"})

In [9]:
#This assigns your vector store as the vector_store to the model later
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [10]:
#This creates an index from your collection, which can be queried later
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

In [ ]:
#Check if it's there!
index

In [17]:
#This specifies the model which you want to use for querying. 
llm=OpenAI(model="gpt-4", temperature=0, max_tokens=1500)

#Llama-index has different chat modes. I like react mode, because you can watch the model reason. For other chat modes, please check their website (there are faster ones).
chat_engine = index.as_chat_engine(
    chat_mode="react",
    llm=llm,
    system_prompt=(
        "Your system instruction." #Specify your system instruction here. Our system instruction can be found in our publication.
    ),
    verbose=True,
    chunk_size = 1048,
    chunk_overlap = 50,
)

In [17]:
#Optional: this resets your chat engine (if you run into context or memory issues...) without re-running everything.
#chat_engine.reset()

In [ ]:
#This queries your chat engine and gives you the answer, the source node and the file name of the document which was queried.
response = chat_engine.chat("Ask me anything.") #Insert your question prompt here. Our prompts can be found in our publication.
from llama_index.core.response.pprint_utils import pprint_response, pprint_source_node
pprint_response(response, show_source=True)
print(response.source_nodes[0].metadata['file_name'])

In [ ]:
#The authors do not authorize the use of this application in clinical practice and are not liable for effects of unauthorized application.